# Obtain a Raman spectrum from BigDFT

This notebook presents how to obtain a Raman spectrum from a set BigDFT calculations. The goal of these calculations is to find the normal modes of vibration of the system under consideration and their intensities. A good reference for the underlying theory of molecular vibrations is the book *Molecular Vibrations* by Wilson *et al.* (1955) or the Advances in Chemical Physics, vol. 67 by K.P. Lawley (where the conversion between atomic units, SI units and the units used in the litterature is well explained).

To get the **energies** of the Raman spectrum, one needs to find the **eigenvalues of the dynamical matrix**, that is closely related to the Hessian. To build these matrices, one must find the derivatives of the forces when each coordinate of each atom is translated by a small amount around the equilibrium positions. To get a better precision on the derivative, each coodinate is translated positively and negatively, so that the number of BigDFT calculations amounts to $2 * 3 n_{at}$, where $n_{at}$ is the number of atoms (3 for the coordinates ($x$, $y$ and $z$), 2 for the number of calculations per coordinates).

Finally, to get the **intensities** (or **activities**) of the spectrum, one must compute the **derivative of the polarizability tensor along the normal modes**. To that end, one must compute the polarizability tensor at each of the positons used to get the vibrational energies, and this means applying an external electric field along each coordinate. Again, two calculations per coordinate lead to 6 extra calculations, meaning that $36 n_{at}$ additional BigDFT standard calculations are required to obtain a Raman spectrum intensities, leading to 42 $n_{at}$ calculations in total.


For more details on how all these quantities are computed, you may read [A. Stirling, *J. Chem. Phys.* **104**, 1254 (1996)](http://dx.doi.org/10.1063/1.470783) and [D. Porezag and M. R.Pederson, *PRB* **54**, 7830-7836 (1996)](https://link.aps.org/doi/10.1103/PhysRevB.54.7830). To check for the polarizability tensor and, more importantly, its derivative, we used [K. M. Gough *et al.*, *Can. J. Chem.* **74**, 1139-1144 (1996)](https://doi.org/10.1139/v96-128) as a comparison.

We will consider the cases of the nitrogen molecule N$_2$ and the water molecule H$_2$O. 

**WARNING**: The current implementation allows the calculations of Raman spectra of systems made of atoms from H to Ne only. To bypass this limitation, you need to add the mass of the other atoms directly in the code (currently, it amounts to modify the dictionary of mass MASS_ATOMS in ramanspectrum.py).

**Import classes and modules**

Remember to [prepare the environment](http://bigdft.org/Wiki/index.php?title=Prepare_the_environment) before launching the notebook.

In [1]:
from __future__ import print_function
from ramanspectrum import RamanSpectrumCalc
from inputfiles import Input, Posinp

# RamanSpectrumCalc class

To get the Raman spectrum for a given system, you may use the `RamanSpectrumCalc`. The main parameters of this class are :

* Main arguments:

    * a BigDFT input file that is an instance of the inputfiles.Input class.
    
    * a initial geometry file (or posinp) that is an instance of the inputfiles.Posinp class.
    
    * `alpha_x` (default value set to 1/64), which is a multiplying factor used to define the displacements of the atoms in each direction. The displacements in each direction are defined by the product of the grid step hgrid (that is usually defined in the input file) and alpha_x.
    
    * `calc_intensities` (default value set to `True`), which is a boolean setting if the intensities associated to the normal modes have to be calculated (if True, then the 36 $n_{at}$ extra calculations mentioned in the introduction are performed).
    
    * `ef_amplitudes` (default value set to `[1.E-4]*3`), which corresponds to the amplitudes of the electric fields along each directions. If `calc_intensities` is set to `True`, is is used to compute the polarizability tensor for each out of equilibrium geometries. This value must not be too high, otherwise some occupied orbitals may have positive energies, and this would lead to incorrect intensities, because the simulated system is no longer the one that is expected (the harmonic approximation would break).
    
The first two main arguments are positional and come from the BigDFTCalc class (that is the parent of the RamanSpectrumCalc class) while the others are specific to the RamanSpectrumCalc class.

* Optional arguments:

    * `prefix`: mainly used as the prefix of the input files,
    
    * `run_folder`: folder where to run the calculation,
    
    * `ref_calc`: instance of the BigDFTCalc class used as a reference.
    
These three arguments actually come from the BigDFTCalc class.

# Case of the N$_2$ molecule

As a first example, let us study the Raman spectrum of the N$_2$ molecule.

### Input file

We give the reference input yaml file as a string. It sets the grid, the convergence parameter and the exchange correlation used for the DFT calculation.

In [2]:
N2_ref_yaml = """\
 dft:
   hgrids: [0.45, 0.45, 0.45]
   rmult: [5.0, 7.0]
   gnrm_cv: 1.e-4
   ixc: LDA"""

We have to convert it to an Input instance in order to use it:

In [3]:
N2_input_yaml = Input.from_string(N2_ref_yaml)
print(N2_input_yaml)

{'dft': {'rmult': [5.0, 7.0], 'gnrm_cv': 0.0001, 'hgrids': [0.45, 0.45, 0.45], 'ixc': 'LDA'}}


### Equilibrium geometry

**Before running a Raman spectrum calulation, you must have performed a geometry optimization to obtain the reference equilibrium positions of that system.**

However, the goal of this notebook is not to perform a geometry optimization using BigDFT, so the posinp file obtained after a precise geometry optimization (using `forcemax: 1.E-6`) is reproduced here as a string:

In [4]:
N2_ref_posinp = """2  angstroem
free
N    2.976307744763e-23    6.872205902435e-23    1.071620018790e-02
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769e+00"""

It is finally converted into a Posinp instance.

In [5]:
N2_posinp = Posinp.from_string(N2_ref_posinp)
print(N2_posinp)

2   angstroem
free
N    2.976307744763e-23    6.872205902435e-23    0.0107162001879
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769



### Perform the Raman spectrum calculation

Now that the input files are defined, we can initialize the Raman spectrum calculation:

In [6]:
rsc_N2 = RamanSpectrumCalc(N2_input_yaml, N2_posinp, ef_amplitudes=[1.E-3]*3, run_folder="N2", prefix="N2")

Note that:

* We set the `run_folder` to "N2", meaning that all the calculations will be started in the "N2" folder (that will be created). 

* All the calculations will have `N2.yaml` and `N2.xyx` as input files since `prefix` is also set to "N2". 

* We kept the default value of alpha_x.

* We use electric field amplitudes that are an order of magnitude larger than their default values. 

It is now possible to run the calculation, using the `run` method of the RamanSpectrumCalc class. It has three main optional arguments:

* `nmpi`: sets the number of mpi tasks to use for all the calculations,

* `nomp`: sets the number of OpenMP tasks,

* `force_run`: if `True`, then the calclulations are forced to run, even though a previous, finished logfile already existed.

The default values of `nmpi` and `nomp` are 1. We will only set `nomp` to 4, here. **You may change these values to suit your need or compilation options**.

In [7]:
rsc_N2.run(nomp=4)

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/ref
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+/along_x+_0.001
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+/along_x-_0.001
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+/along_y+_0.001
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+/along_y-_0.001
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/N2/atom_000/x+/along_z+_0.001
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/bigd

### Main results

The Raman spectrum is now computed, and we can print the main results, namely the energy of each normal mode, their intensity and their depolarization ratio. All these data are accessible via the attributes `energies`, `intensities` and `depol_ratios` of the current instance.

In [8]:
print("Vibration energies in cm^-1:\n", rsc_N2.energies['cm^-1'])
print("Peak intensities in Ang^4.amu^-1:\n", rsc_N2.intensities)
print("Depolarization ratios:\n", rsc_N2.depol_ratios)

Vibration energies in cm^-1:
 [  0.00000000e+00   2.38953700e+03   2.03443423e+01   2.03443423e+01
   8.53747682e-06   1.98169049e-07]
Peak intensities in Ang^4.amu^-1:
 [8.7235064145752243e-29, 22.131255815876642, 1.1272517675330971, 1.1272517674710711, 1.840376763961839e-31, 8.7230307410188481e-29]
Depolarization ratios:
 [0.74999250133248163, 0.096634912535125059, 0.74999258345589825, 0.74999258340973229, 0.023414838154719626, 0.74999324045756621]


The nitrogen molecule being linear, it has $3 n_{at} - 5 = 1$ normal mode. Its energy, found using the LDA exchange-correlation is around 2390 cm$^{-1}$. Following the statement of [B. G. Johnson *et al.*, *J. Chem. Phys.* **98**, 5612 (1993)](http://aip.scitation.org/doi/10.1063/1.464906), our result should be compared with the harmonic experimental value, which is reported to be 2360 cm$^{-1}$ in this same reference. There is a very good agreement! Note that the actual experimental value (without neglecting anharmocity) is 2331 cm$^{-1}$.

A value of 0.11 for the depolarization ratio was also reported in [P. L. Polavarapu, *J. Chem. Phys.* **94**, 8106-8112 (1990)](http://pubs.acs.org/doi/abs/10.1021/j100384a024), which compares rather well to our 0.097. An energy of 2725 cm$^{-1}$ was also reported, along with an intensity of 26.0 $\unicode[serif]{xC5}^4$ amu$^{-1}$.

# Case of the H$_2$O molecule

Let us now study the H$_2$O molecule, which is, in contrast with N$_2$, dipolar and non-linear.

### Input file

We keep the same input file as for the N$_2$ molecule.

In [9]:
H2O_ref_yaml = """\
 dft:
   hgrids: [0.45, 0.45, 0.45]
   rmult: [5.0, 7.0]
   gnrm_cv: 1.e-5
   ixc: LDA"""

# Convert the string to an Input instance
H2O_input_yaml = Input.from_string(H2O_ref_yaml)
print(H2O_input_yaml)

{'dft': {'rmult': [5.0, 7.0], 'gnrm_cv': 1e-05, 'hgrids': [0.45, 0.45, 0.45], 'ixc': 'LDA'}}


### Equilibrium geometry

Again, the geometry optimization is not performed here. The final posinp a such a calculation is given below:

In [10]:
H2O_ref_posinp = """\
3  angstroemd0  -1.71815715282446995E+01 (Ha)  FINAL CONFIGURATION
free  {format: xyz, source: posinp.xyz}
O  1.49999906977026298E+00  2.04308426947126288E-02  1.50000000000000000E+00
H  7.29039979975805386E-01  6.10703212031191822E-01  1.50000000000000000E+00
H  2.27096106946322118E+00  6.10703923637132329E-01  1.50000000000000000E+00"""

# Convert it to a Posinp instance
H2O_posinp = Posinp.from_string(H2O_ref_posinp)
print(H2O_posinp)

3   angstroemd0
free
O    1.49999906977026    0.0204308426947126    1.5
H    0.729039979975805    0.610703212031192    1.5
H    2.27096106946322    0.610703923637132    1.5



### Perform the Raman spectrum calculation

Now that the input files are defined, we can initialize and then run the Raman spectrum calculation:

In [11]:
rsc_H2O = RamanSpectrumCalc(H2O_input_yaml, H2O_posinp, ef_amplitudes=[1.E-3]*3, run_folder="H2O", prefix="H2O")
rsc_H2O.run(nomp=4)#, force_run=True)

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/ref
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+/along_x+_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+/along_x-_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+/along_y+_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+/along_y-_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O/atom_000/x+/along_z+_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/

### Main results

Let us finally take a look at the results:

In [12]:
print("Vibration energies in cm^-1:\n", rsc_H2O.energies['cm^-1'])
print("Peak intensities in Ang^4.amu^-1:\n", rsc_H2O.intensities)
print("Depolarization ratios:\n", rsc_H2O.depol_ratios)

Vibration energies in cm^-1:
 [  3.46516591e+03+0.j           3.39892849e+03+0.j           1.66271154e+03+0.j
   6.67254256e+01+0.j           1.00165772e+02+0.j           2.80659697e+01+0.j
   3.03327217e-04+0.00022503j   3.03327217e-04-0.00022503j
   1.25731405e-05+0.j        ]
Peak intensities in Ang^4.amu^-1:
 [(103.63128786032038+0j), (81.803186813736062+0j), (1.7594241104189812+0j), (-0.0025998149391265879+2.3501868207822926e-18j), (0.0041610972097172695-1.4994484334980364e-18j), (0.091597981773405404+0j), (0.0021112818615370665+0.0077836410571581116j), (0.0021112818615370665-0.0077836410571581116j), (0.00030356839346627968+0j)]
Depolarization ratios:
 [(0.050957806413779387+0j), (0.07551957233698281+0j), (0.15905122835647573+0j), (-2.7657988183588449+4.1970897530776357e-15j), (0.66457040289665881-8.5113760804651995e-17j), (0.74956712965340788+0j), (0.3232154501435236+0.03239786760566165j), (0.3232154501435236-0.03239786760566165j), (0.33954175041590912+0j)]


The water molecule is not linear, therefore it has $3 n_{at} - 6 = 3$ normal modes, which makes a little more data to look at, while we found more litterature than for the nitrogen molecule. 

The energies, found using the LDA exchange-correlation, are around 1662, 3402 and 3466 cm$^{-1}$. 

From [\[1\]](http://aip.scitation.org/doi/10.1063/1.464906), the harmonic experimental value are reported to be 1648, 3832 and 3943 cm$^{-1}$, while they report LDA energies (column S-VWN in table IV) at 1646, 3657 and 3789 cm$^{-1}$. The agreement of our results is not as good as for the N$_2$ molecule for the two highest energy vibrations. Note that the actual experimental values (without neglecting anharmocity) are 1595, 3657 and 3756 cm$^{-1}$. In [\[2\]](https://link.aps.org/doi/10.1103/PhysRevB.54.7830), the reported LDA energies are 1534, 3698, 3812 cm$^{-1}$.

With regard to the intensities, the experimental values are 0.9, 108 and 19.2 $\unicode[serif]{xC5}^4$ amu$^{-1}$ and LDA values are 1.0, 111, and 25 $\unicode[serif]{xC5}^4$ amu$^{-1}$ (from [\[3\]](http://dx.doi.org/10.1063/1.470783)), while [2] reports 0.632, 115 and 24.8 $\unicode[serif]{xC5}^4$ amu$^{-1}$.

The experimental depolarization ratio are 0.74, 0.03 and 0.75, while LDA values are reported in [3] to be 0.59, 0.04, 0.75.

You may also find more values for the energies, intensities and depolarization ratio for the water molecule using other methods or exchange-correlation in [2-5].

As you can see, the results we obtained are not in a correct agreement with the litterature for these three quantities, and we will show in the following that this requires a denser, larger grid as well as a smaller convergence parameter for the BigDFT calculations to be considered as converged.

[1] B. G. Johnson *et al.*, *J. Chem. Phys.* **98**, 5612 (1993)

[2] D. Porezag and M. R.Pederson, *PRB* **54**, 7830-7836 (1996)

[3] A. Stirling, *J. Chem. Phys.* **104**, 1254 (1996)

\[4\] [E. E. Fileti *et al*, *Chem. Phys. Lett.* **452**, 54-58 (2008)](https://doi.org/10.1016/j.cplett.2007.12.059)

[5] K. P. Lawley, Advances in Chemical Physics vol. 67 (2009)

## More refined calculation for the water molecule

Let us perform a little more demanding calculations to get converged results.

### Input file

We decease the grid spaces `hgrids` along the three directions and the convergence parameter `gnrm_cv` while increasing the grid extensions `rmult`.

In [13]:
H2O_better_ref_yaml = """\
 dft:
   hgrids: [0.30, 0.30, 0.30]
   rmult: [9.0, 9.0]
   gnrm_cv: 1.e-5
   ixc: LDA"""

# Convert the string to an Input instance
H2O_better_input_yaml = Input.from_string(H2O_better_ref_yaml)
print(H2O_better_input_yaml)

{'dft': {'rmult': [9.0, 9.0], 'gnrm_cv': 1e-05, 'hgrids': [0.3, 0.3, 0.3], 'ixc': 'LDA'}}


### Perform the Raman spectrum calculation

We keep the same posinp as earlier to initialize and then run the Raman spectrum calculation (in another folder):

In [14]:
rsc_H2O_better = RamanSpectrumCalc(H2O_better_input_yaml, H2O_posinp, ef_amplitudes=[1.E-3]*3, \
                            run_folder="H2O_better", prefix="H2O")
rsc_H2O_better.run(nomp=4)

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/ref
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+/along_x+_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+/along_x-_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+/along_y+_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+/along_y-_0.001
Logfile log-H2O.yaml already exists!

/Users/maximemoriniere/Documents/bigdft/BigDFT-nb/Pol-Tensor_Raman-spectrum/H2O_better/atom_000/x+/along_z+_0.001
Logfile log-

### Main results

The results now are in better, if not excellent, agreement with the litterature:

In [15]:
print("Vibration energies in cm^-1:\n", rsc_H2O_better.energies['cm^-1'])
print("Peak intensities in Ang^4.amu^-1:\n", rsc_H2O_better.intensities)
print("Depolarization ratios:\n", rsc_H2O_better.depol_ratios)

Vibration energies in cm^-1:
 [  3.82485622e+03+0.j           3.68903540e+03+0.j           1.47274046e+03+0.j
   2.65664443e+01+0.j           5.13779526e+01+0.j           2.43856421e+01+0.j
   2.25515713e-04+0.00020367j   2.25515713e-04-0.00020367j
   1.18832227e-05+0.j        ]
Peak intensities in Ang^4.amu^-1:
 [(25.16873684293418+0j), (118.75292258533018+0j), (0.46179639360468761+0j), (-0.0013687910702543551+4.2516714694349167e-19j), (8.9568350022466374e-06+0j), (0.015955015989412322+0j), (0.010091566368728069+0.00059163009009740545j), (0.010091566368728069-0.00059163009009740545j), (0.0001797590669122242+0j)]
Depolarization ratios:
 [(0.74980498229165804+0j), (0.040668878289683179+0j), (0.29347825046830606+0j), (1.1651089352733124+3.1706639086369708e-16j), (0.33736821120124438+0j), (0.74999998905387255+0j), (0.3029128034416475-0.028594842034250865j), (0.3029128034416475+0.028594842034250865j), (0.32124632387549967+0j)]


With this better grid, the energies found using the LDA exchange-correlation are around 1473, 3689 and 3825 cm$^{-1}$. As a reminder, the harmonic experimental value are reported to be 1648, 3832 and 3943 cm$^{-1}$, while LDA energies are reported to be at 1646, 3657 and 3789 cm$^{-1}$ and 1534, 3698, 3812 cm$^{-1}$ in two different articles. The agreement with the other LDA calculations therefore requires a good convergence with respect to the wavelet grid.

The same statement is valid for the intensities: we report 0.46, 119 and 25 $\unicode[serif]{xC5}^4$ amu$^{-1}$, to be compared with other LDA values of 1.0, 111, and 25 $\unicode[serif]{xC5}^4$ amu$^{-1}$ and 0.632, 115 and 24.8 $\unicode[serif]{xC5}^4$ amu$^{-1}$, while the experimental values are $0.9\pm 2$, $108 \pm 14$ and $19.2 \pm 2.1$ $\unicode[serif]{xC5}^4$ amu$^{-1}$.

With regard to the depolarization ratio, LDA values are reported to be 0.59, 0.04, 0.75, which is in very good agreement with the values we obtained for the two highest excitations.